In [15]:
# from datasets import load_dataset

# dataset = load_dataset("yelp_review_full")

In [16]:
import pickle
# with open('yelp_dataset.pkl', 'wb') as file:
#     pickle.dump(dataset, file)

In [17]:
with open('yelp_dataset.pkl', 'rb') as file:
    dataset = pickle.load(file)

In [18]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})


In [19]:
train_dataset = dataset['train']
test_dataset = dataset['test']

# Convert train and test datasets to arrays
train_data = train_dataset['text']
train_labels = train_dataset['label']
test_data = test_dataset['text']
test_labels = test_dataset['label']

# Convert labels to lists (optional)
# train_labels = train_labels.tolist()
# test_labels = test_labels.tolist()

In [20]:
print(train_data[0])
print(train_labels[0])


dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
4


## Preprocess

In [21]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sriteja/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sriteja/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
import sys  
!{sys.executable} -m pip install contractions
import contractions

In [23]:
# import re  # Import the regular expressions module

# # use tokenizer to remove punctuation

# def tokenize_text(text):
#     # Replace "!" with "exm" using regular expressions
#     text = re.sub(r'!', ' exm', text)
    
#     expanded_words = []
#     for word in text.split():
#         # using contractions.fix to expand the shortened words
#         expanded_words.append(contractions.fix(word))   
    
#     expanded_text = ' '.join(expanded_words)
#     # print(expanded_text)
    
#     text = expanded_text    
    
#     # replace a-b with a and b
#     text = text.replace('-', ' ')
    
#     tokens = nltk.word_tokenize(text)
    
#     # Add an extra occurrence for all-uppercase words with more than one letter
#     # temp = [word if (len(word) > 1 and word.isupper()) else None for word in tokens]
#     temp = []
#     for word in tokens:
#         if len(word) > 1 and word.isupper():
#             temp.append(word)
#     tokens.extend(x for x in temp if x)
    
#     # convert to lower case
#     tokens = [w.lower() for w in tokens]
#     # dr. = dr and st. = st and so on
#     tokens = [w.replace('.', '') for w in tokens]
    
#     # remove punctuation
#     # tokens = [word for word in tokens if word.isalpha()]
    
#     # Remove stop words
#     stop_words = set(stopwords.words("english"))
#     tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    
#     return tokens

# # print(train_data[4])
# # tokenize_text(train_data[4])


In [24]:
# tokens = []
# print(len(train_data))

# for i in range(len(train_data)):
#     tokens.append(tokenize_text(train_data[i]))
#     if i % 1000 == 0:
#         print(i)

# # save to pkl file
# with open('yelp_train_tokens_no_stop.pkl', 'wb') as file:
#     pickle.dump(tokens, file)

In [25]:
# import pickle
# from collections import Counter

# # Load the tokenized data
# with open('yelp_train_tokens_no_stop.pkl', 'rb') as file:
#     tokens = pickle.load(file)

# # Define the frequency cutoff threshold
# frequency_cutoff = 10

# # Count word frequencies
# word_counts = Counter(word for tokens_list in tokens for word in tokens_list)

# # Filter out words with counts less than the threshold
# filtered_tokens = [[word for word in tokens_list if word_counts[word] >= frequency_cutoff] for tokens_list in tokens]

# # Save the filtered tokens to a new file
# with open('yelp_train_tokens_filtered.pkl', 'wb') as file:
#     pickle.dump(filtered_tokens, file)

In [26]:
# tokens_test = []
# print(len(test_data))

# for i in range(len(test_data)):
#     tokens_test.append(tokenize_text(test_data[i]))
#     if i % 1000 == 0:
#         print(i)

# with open('yelp_test_tokens_no_stop.pkl', 'wb') as file:
#     pickle.dump(tokens_test, file)

In [27]:
# import pickle
# from collections import Counter

# # Load the tokenized data
# with open('yelp_test_tokens_no_stop.pkl', 'rb') as file:
#     tokens = pickle.load(file)

# # Define the frequency cutoff threshold
# frequency_cutoff = 10

# # Count word frequencies
# word_counts = Counter(word for tokens_list in tokens for word in tokens_list)

# # Filter out words with counts less than the threshold
# filtered_tokens = [[word for word in tokens_list if word_counts[word] >= frequency_cutoff] for tokens_list in tokens]

# # Save the filtered tokens to a new file
# with open('yelp_test_tokens_filtered.pkl', 'wb') as file:
#     pickle.dump(filtered_tokens, file)

In [28]:
import numpy as np

train_tokens = []
with open('yelp_train_tokens_filtered.pkl', 'rb') as file:
    train_tokens = pickle.load(file)

test_tokens = []
with open('yelp_test_tokens_filtered.pkl', 'rb') as file:
    test_tokens = pickle.load(file)
    
print("Loaded tokens")



Loaded tokens


In [29]:
# in train data replace every 1000th word with UNK randomly

import random

for i in range(len(train_tokens)):
    for j in range(len(train_tokens[i])):
        if random.randint(1, 1000) == 1:
            train_tokens[i][j] = 'UNK'
        

In [30]:
# Build the BoW representation manually
# Create a vocabulary by collecting unique words from the training data
vocab = set()
for tokens in train_tokens:
    vocab.update(tokens)

# Create a dictionary to map words to indices in the vocabulary
vocab_dict = {}
for i, word in enumerate(vocab):
    vocab_dict[word] = i

print(len(vocab_dict))
# Initialize BoW matrices for training and testing data


45287


In [31]:
# reduced_train_tokens = train_tokens

# # Build the BoW representation manually
# # Create a vocabulary by collecting unique words from the training data
# vocab_reduced = set()
# for tokens in reduced_train_tokens:
#     vocab_reduced.update(tokens)

# # Create a dictionary to map words to indices in the vocabulary
# vocab_dict_reduced = {}
# for i, word in enumerate(vocab_reduced):
#     vocab_dict_reduced[word] = i

# print(len(vocab_dict_reduced))

# train_bow = np.zeros((len(reduced_train_tokens), len(vocab_reduced)))

# print("bow train")
# # Convert text to BoW vectors
# for i, tokens in enumerate(reduced_train_tokens):
#     if i % 5000 == 0:
#         print(i)
#     for token in tokens:
#         train_bow[i][vocab_dict_reduced[token]] += 1



# reduced_test_tokens = test_tokens[:5000]
# test_bow = np.zeros((len(reduced_test_tokens), len(vocab_reduced)))

# print("bow test")

# for i, tokens in enumerate(reduced_test_tokens):
#     if i % 1000 == 0:
#         print(i)
#     for token in tokens:
#         if token in vocab_dict_reduced:
#             test_bow[i][vocab_dict_reduced[token]] += 1
#         else:    # if there is an unknown word, add it to the UNK column 
#             test_bow[i][vocab_dict_reduced['UNK']] += 1

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 10
learning_rate = 0.05
batch_size = 5000

class BoWClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(BoWClassifier, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        # Pass the input through the linear layer
        out = self.linear(x)
        return out
    
# Define the model
input_size = len(vocab)  # Input size is the size of the vocabulary
output_size = 5  # Output size is 5 dimensions

model = BoWClassifier(input_size, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Training loop
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i in range(0, len(train_tokens), batch_size):
        
        inputs = train_tokens[i:i + batch_size]

        # inputs = inputs.to(device)
        # print(len(inputs))
        # print(len(vocab))
        
        # make bow vector for inputs
        bow = torch.zeros((batch_size, len(vocab)), dtype=torch.float32).to(device)
        # print(bow.shape)
        for j in range(batch_size):
            # token_indices = [vocab_dict[token] for token in train_tokens[i + j]]
            # bow[j, token_indices] = torch.tensor([train_tokens[i + j].count(token) for token in vocab_dict.keys() if token in train_tokens[i + j]]).float().to(device)

            for token in inputs[j]:        
                bow[j][vocab_dict[token]] += 1
    
        
        # convert bow to tensor
        # inputs = torch.tensor(bow, dtype=torch.float32)
        inputs = bow.to(device)
        
        labels = train_labels[i:i + batch_size]  # Make sure to have train_labels defined
        # Convert labels to LongTensors
        labels = torch.tensor(labels, dtype=torch.long)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate the loss
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()

        # Update weights
        optimizer.step()
        
        # calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Print the loss for this batch if needed
        print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i//batch_size+1}], Loss: {loss.item()}')
        
    print(f'Accuracy of the network on the {total} train inputs: {100 * correct / total} %')

print('Training finished')

Epoch [1/10], Batch [1], Loss: 1.6094996929168701
Epoch [1/10], Batch [2], Loss: 1.5401923656463623
Epoch [1/10], Batch [3], Loss: 1.4586058855056763
Epoch [1/10], Batch [4], Loss: 1.3866132497787476
Epoch [1/10], Batch [5], Loss: 1.3490192890167236
Epoch [1/10], Batch [6], Loss: 1.3084720373153687
Epoch [1/10], Batch [7], Loss: 1.2692068815231323
Epoch [1/10], Batch [8], Loss: 1.2505382299423218
Epoch [1/10], Batch [9], Loss: 1.2511183023452759
Epoch [1/10], Batch [10], Loss: 1.2164318561553955
Epoch [1/10], Batch [11], Loss: 1.2010159492492676
Epoch [1/10], Batch [12], Loss: 1.1869926452636719
Epoch [1/10], Batch [13], Loss: 1.1876052618026733
Epoch [1/10], Batch [14], Loss: 1.176352858543396
Epoch [1/10], Batch [15], Loss: 1.1256908178329468
Epoch [1/10], Batch [16], Loss: 1.207547664642334
Epoch [1/10], Batch [17], Loss: 1.1719365119934082
Epoch [1/10], Batch [18], Loss: 1.1670870780944824
Epoch [1/10], Batch [19], Loss: 1.170340657234192
Epoch [1/10], Batch [20], Loss: 1.154589056

In [ ]:
# save model
torch.save(model.state_dict(), 'lin_layer_01.pth')

# save model as pkl file
torch.save(model, 'lin_layer_01.pkl')

# save model as pkl file
# torch.save(model,)